https://www.kaggle.com/san-francisco/sf-street-tree-list?select=street-tree-list.csv

In [1]:
import petl as etl

In [5]:
trees_legal_status = [['Number', 'TreeID', 'LegalStatus'],
                      [1, 46534, 'Permitted Site'],
                      [2, 121399, 'DPW Maintained'],
                      [3, 85269, 'Permitted Site'],
                      [4, 254266, 'Permitted Site'],
                      [5, 20000, 'DPW Maintained'],
                      [6, 254265, 'Permitted Site'],
                      [7, 53918, 'Permitted Site'],
                      [8, 19600, 'DPW Maintained']]

In [6]:
street_trees_info = [['Number', 'TreeID', 'Address', 'SiteOrder', 'Caretaker'],
                     [1, 46534, '73 Summer St', 7, 'Private'],
                     [2, 121399, '349X Cargo Way', 1, 'DPW'],
                     [3, 85269, '1000 Edinburgh St', 3, 'Private'],
                     [5, 20000, '100 STAIRWAY46', 20, 'DPW'],
                     [6, 254265, '85X Sussex St', 1, 'Private'],
                     [7, 53918, '3 21st St', 1, 'Private'],
                     [9, 121227, '4299x 17th St', 3, 'DPW']]

Default join

In [7]:
default_join = etl.join(trees_legal_status, 
                        street_trees_info)

#### Only matching rows are present
Trees #4 and #8 from the trees_legal_status table and #9 from the street_trees_info table are missing

In [12]:
default_join.lookall()

+--------+--------+------------------+---------------------+-----------+-----------+
| Number | TreeID | LegalStatus      | Address             | SiteOrder | Caretaker |
+========+========+==================+=====================+===========+===========+
|      1 |  46534 | 'Permitted Site' | '73 Summer St'      |         7 | 'Private' |
+--------+--------+------------------+---------------------+-----------+-----------+
|      2 | 121399 | 'DPW Maintained' | '349X Cargo Way'    |         1 | 'DPW'     |
+--------+--------+------------------+---------------------+-----------+-----------+
|      3 |  85269 | 'Permitted Site' | '1000 Edinburgh St' |         3 | 'Private' |
+--------+--------+------------------+---------------------+-----------+-----------+
|      5 |  20000 | 'DPW Maintained' | '100 STAIRWAY46'    |        20 | 'DPW'     |
+--------+--------+------------------+---------------------+-----------+-----------+
|      6 | 254265 | 'Permitted Site' | '85X Sussex St'     |     

In [13]:
etl.nrows(default_join)

6

Perform an equi-join on the given tables

In [14]:
equi_join = etl.join(trees_legal_status, 
                     street_trees_info, 
                     key = 'Number')

In [15]:
equi_join.lookall()

+--------+--------+------------------+--------+---------------------+-----------+-----------+
| Number | TreeID | LegalStatus      | TreeID | Address             | SiteOrder | Caretaker |
+========+========+==================+========+=====================+===========+===========+
|      1 |  46534 | 'Permitted Site' |  46534 | '73 Summer St'      |         7 | 'Private' |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      2 | 121399 | 'DPW Maintained' | 121399 | '349X Cargo Way'    |         1 | 'DPW'     |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      3 |  85269 | 'Permitted Site' |  85269 | '1000 Edinburgh St' |         3 | 'Private' |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      5 |  20000 | 'DPW Maintained' |  20000 | '100 STAIRWAY46'    |        20 | 'DPW'     |
+--------+--------+------------------+--------+-------------

In [16]:
etl.nrows(equi_join)

6

Alternative implementation of petl.transform.joins.join(), where the join is executed by constructing an in-memory lookup for the right hand table, then iterating over rows from the left hand table.

In [18]:
hash_join = etl.hashjoin(trees_legal_status, 
                         street_trees_info, 
                         key = 'Number')

In [19]:
hash_join.lookall()

+--------+--------+------------------+--------+---------------------+-----------+-----------+
| Number | TreeID | LegalStatus      | TreeID | Address             | SiteOrder | Caretaker |
+========+========+==================+========+=====================+===========+===========+
|      1 |  46534 | 'Permitted Site' |  46534 | '73 Summer St'      |         7 | 'Private' |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      2 | 121399 | 'DPW Maintained' | 121399 | '349X Cargo Way'    |         1 | 'DPW'     |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      3 |  85269 | 'Permitted Site' |  85269 | '1000 Edinburgh St' |         3 | 'Private' |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      5 |  20000 | 'DPW Maintained' |  20000 | '100 STAIRWAY46'    |        20 | 'DPW'     |
+--------+--------+------------------+--------+-------------

In [20]:
etl.nrows(hash_join)

6

#### Perform a left outer join on the given tables
Trees #4 and #8 show up with None values for the fields in the street_trees_info table since there is no corresponding record there

In [22]:
left_join = etl.leftjoin(trees_legal_status, 
                         street_trees_info, 
                         key = 'TreeID')

In [23]:
etl.lookall(left_join)

+--------+--------+------------------+--------+---------------------+-----------+-----------+
| Number | TreeID | LegalStatus      | Number | Address             | SiteOrder | Caretaker |
+========+========+==================+========+=====================+===========+===========+
|      8 |  19600 | 'DPW Maintained' | None   | None                | None      | None      |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      5 |  20000 | 'DPW Maintained' |      5 | '100 STAIRWAY46'    |        20 | 'DPW'     |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      1 |  46534 | 'Permitted Site' |      1 | '73 Summer St'      |         7 | 'Private' |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      7 |  53918 | 'Permitted Site' |      7 | '3 21st St'         |         1 | 'Private' |
+--------+--------+------------------+--------+-------------

In [24]:
etl.nrows(left_join)

8

#### When the join keys have different names in the left and right tables
- use the same trees_legal_status table, but a different one for street_trees_info called street_trees_alt
- the difference is that street_trees_alt has TreeNumber instead of number

In [38]:
trees_legal_status = [['Number', 'TreeID', 'LegalStatus'],
                      [1, 46534, 'Permitted Site'],
                      [2, 121399, 'DPW Maintained'],
                      [3, 85269, 'Permitted Site'],
                      [4, 254266, 'Permitted Site'],
                      [5, 20000, 'DPW Maintained'],
                      [6, 254265, 'Permitted Site'],
                      [7, 53918, 'Permitted Site'],
                      [8, 19600, 'DPW Maintained']]

In [39]:
street_trees_alt = [['TreeNumber', 'TreeID', 'Address', 'SiteOrder', 'Caretaker'],
                    [1, 46534, '73 Summer St', 7, 'Private'],
                    [2, 121399, '349X Cargo Way', 1, 'DPW'],
                    [3, 85269, '1000 Edinburgh St', 3, 'Private'],
                    [5, 20000, '100 STAIRWAY46', 20, 'DPW'],
                    [6, 254265, '85X Sussex St', 1, 'Private'],
                    [7, 53918, '3 21st St', 1, 'Private'],
                    [9, 121227, '4299x 17th St', 3, 'DPW']]

#### The normal join does a match based on the common TreeNumber columns

In [40]:
default_join = etl.join(trees_legal_status, 
                        street_trees_alt)

default_join.lookall()

+--------+--------+------------------+------------+---------------------+-----------+-----------+
| Number | TreeID | LegalStatus      | TreeNumber | Address             | SiteOrder | Caretaker |
+========+========+==================+============+=====================+===========+===========+
|      5 |  20000 | 'DPW Maintained' |          5 | '100 STAIRWAY46'    |        20 | 'DPW'     |
+--------+--------+------------------+------------+---------------------+-----------+-----------+
|      1 |  46534 | 'Permitted Site' |          1 | '73 Summer St'      |         7 | 'Private' |
+--------+--------+------------------+------------+---------------------+-----------+-----------+
|      7 |  53918 | 'Permitted Site' |          7 | '3 21st St'         |         1 | 'Private' |
+--------+--------+------------------+------------+---------------------+-----------+-----------+
|      3 |  85269 | 'Permitted Site' |          3 | '1000 Edinburgh St' |         3 | 'Private' |
+--------+--------+-

#### As does leftjoin

In [41]:
left_join = etl.leftjoin(trees_legal_status, 
                         street_trees_alt)

left_join.lookall()

+--------+--------+------------------+------------+---------------------+-----------+-----------+
| Number | TreeID | LegalStatus      | TreeNumber | Address             | SiteOrder | Caretaker |
+========+========+==================+============+=====================+===========+===========+
|      8 |  19600 | 'DPW Maintained' | None       | None                | None      | None      |
+--------+--------+------------------+------------+---------------------+-----------+-----------+
|      5 |  20000 | 'DPW Maintained' |          5 | '100 STAIRWAY46'    |        20 | 'DPW'     |
+--------+--------+------------------+------------+---------------------+-----------+-----------+
|      1 |  46534 | 'Permitted Site' |          1 | '73 Summer St'      |         7 | 'Private' |
+--------+--------+------------------+------------+---------------------+-----------+-----------+
|      7 |  53918 | 'Permitted Site' |          7 | '3 21st St'         |         1 | 'Private' |
+--------+--------+-

#### Rename the TreeID column so that it's called ID
Now there is no common column between trees_legal_status and street_trees_alt

In [42]:
street_trees_alt = [['TreeNumber', 'ID', 'Address', 'SiteOrder', 'Caretaker'],
                    [1, 46534, '73 Summer St', 7, 'Private'],
                    [2, 121399, '349X Cargo Way', 1, 'DPW'],
                    [3, 85269, '1000 Edinburgh St', 3, 'Private'],
                    [5, 20000, '100 STAIRWAY46', 20, 'DPW'],
                    [6, 254265, '85X Sussex St', 1, 'Private'],
                    [7, 53918, '3 21st St', 1, 'Private'],
                    [9, 121227, '4299x 17th St', 3, 'DPW']]

#### This join throws an error

In [43]:
default_join = etl.join(trees_legal_status, 
                        street_trees_alt)

default_join.lookall()

AssertionError: no fields in common

#### As does left join again

In [44]:
left_join = etl.leftjoin(trees_legal_status, 
                         street_trees_alt)

left_join.lookall()

AssertionError: no fields in common

#### Use the lkey and rkey parameters when there is no common column name
- Doing a regular join using Number/TreeNumber
- The TreeNumber from the right table does not appear in the result, since it has the same value as Number in the left table

In [45]:
default_join = etl.join(trees_legal_status, 
                        street_trees_alt,
                          lkey = 'Number', 
                          rkey = 'TreeNumber')

default_join.lookall()

+--------+--------+------------------+--------+---------------------+-----------+-----------+
| Number | TreeID | LegalStatus      | ID     | Address             | SiteOrder | Caretaker |
+========+========+==================+========+=====================+===========+===========+
|      1 |  46534 | 'Permitted Site' |  46534 | '73 Summer St'      |         7 | 'Private' |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      2 | 121399 | 'DPW Maintained' | 121399 | '349X Cargo Way'    |         1 | 'DPW'     |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      3 |  85269 | 'Permitted Site' |  85269 | '1000 Edinburgh St' |         3 | 'Private' |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      5 |  20000 | 'DPW Maintained' |  20000 | '100 STAIRWAY46'    |        20 | 'DPW'     |
+--------+--------+------------------+--------+-------------

#### Doing a left join using TreeID/ID

In [46]:
left_join = etl.leftjoin(trees_legal_status, 
                         street_trees_alt,
                         lkey = 'TreeID', 
                         rkey = 'ID'
                        )

left_join.lookall()

+--------+--------+------------------+------------+---------------------+-----------+-----------+
| Number | TreeID | LegalStatus      | TreeNumber | Address             | SiteOrder | Caretaker |
+========+========+==================+============+=====================+===========+===========+
|      8 |  19600 | 'DPW Maintained' | None       | None                | None      | None      |
+--------+--------+------------------+------------+---------------------+-----------+-----------+
|      5 |  20000 | 'DPW Maintained' |          5 | '100 STAIRWAY46'    |        20 | 'DPW'     |
+--------+--------+------------------+------------+---------------------+-----------+-----------+
|      1 |  46534 | 'Permitted Site' |          1 | '73 Summer St'      |         7 | 'Private' |
+--------+--------+------------------+------------+---------------------+-----------+-----------+
|      7 |  53918 | 'Permitted Site' |          7 | '3 21st St'         |         1 | 'Private' |
+--------+--------+-

#### Right joins
- Switching back to the original tables with the common columns
- Since there are two potential join keys, we specify which one shoule be used

In [48]:
right_join = etl.rightjoin(trees_legal_status, 
                           street_trees_info, 
                           key = 'TreeID')

In [49]:
etl.lookall(right_join)

+--------+--------+------------------+--------+---------------------+-----------+-----------+
| Number | TreeID | LegalStatus      | Number | Address             | SiteOrder | Caretaker |
+========+========+==================+========+=====================+===========+===========+
|      5 |  20000 | 'DPW Maintained' |      5 | '100 STAIRWAY46'    |        20 | 'DPW'     |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      1 |  46534 | 'Permitted Site' |      1 | '73 Summer St'      |         7 | 'Private' |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      7 |  53918 | 'Permitted Site' |      7 | '3 21st St'         |         1 | 'Private' |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      3 |  85269 | 'Permitted Site' |      3 | '1000 Edinburgh St' |         3 | 'Private' |
+--------+--------+------------------+--------+-------------

In [50]:
outer_join = etl.outerjoin(trees_legal_status, 
                           street_trees_info, 
                           key = 'TreeID')

In [51]:
etl.lookall(outer_join)

+--------+--------+------------------+--------+---------------------+-----------+-----------+
| Number | TreeID | LegalStatus      | Number | Address             | SiteOrder | Caretaker |
+========+========+==================+========+=====================+===========+===========+
|      8 |  19600 | 'DPW Maintained' | None   | None                | None      | None      |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      5 |  20000 | 'DPW Maintained' |      5 | '100 STAIRWAY46'    |        20 | 'DPW'     |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      1 |  46534 | 'Permitted Site' |      1 | '73 Summer St'      |         7 | 'Private' |
+--------+--------+------------------+--------+---------------------+-----------+-----------+
|      7 |  53918 | 'Permitted Site' |      7 | '3 21st St'         |         1 | 'Private' |
+--------+--------+------------------+--------+-------------

Return rows from the left table where the key value does not occur in the right table

In [52]:
anti_join = etl.antijoin(trees_legal_status, 
                         street_trees_info, 
                         key = 'TreeID')

In [53]:
etl.lookall(anti_join)

+--------+--------+------------------+
| Number | TreeID | LegalStatus      |
+========+========+==================+
|      8 |  19600 | 'DPW Maintained' |
+--------+--------+------------------+
|      4 | 254266 | 'Permitted Site' |
+--------+--------+------------------+

In [54]:
address_info, caretaker = etl.unjoin(street_trees_info, 
                                     'Caretaker', 
                                     key = 'TreeID')

In [56]:
address_info.lookall()

+--------+--------+---------------------+-----------+
| Number | TreeID | Address             | SiteOrder |
+========+========+=====================+===========+
|      1 |  46534 | '73 Summer St'      |         7 |
+--------+--------+---------------------+-----------+
|      2 | 121399 | '349X Cargo Way'    |         1 |
+--------+--------+---------------------+-----------+
|      3 |  85269 | '1000 Edinburgh St' |         3 |
+--------+--------+---------------------+-----------+
|      5 |  20000 | '100 STAIRWAY46'    |        20 |
+--------+--------+---------------------+-----------+
|      6 | 254265 | '85X Sussex St'     |         1 |
+--------+--------+---------------------+-----------+
|      7 |  53918 | '3 21st St'         |         1 |
+--------+--------+---------------------+-----------+
|      9 | 121227 | '4299x 17th St'     |         3 |
+--------+--------+---------------------+-----------+

In [57]:
caretaker.lookall()

+--------+-----------+
| TreeID | Caretaker |
+========+===========+
|  20000 | 'DPW'     |
+--------+-----------+
|  46534 | 'Private' |
+--------+-----------+
|  53918 | 'Private' |
+--------+-----------+
|  85269 | 'Private' |
+--------+-----------+
| 121227 | 'DPW'     |
+--------+-----------+
| 121399 | 'DPW'     |
+--------+-----------+
| 254265 | 'Private' |
+--------+-----------+

#### when the key is not unique in either or both tables

In [58]:
trees_site_permit = [['SiteOrder', 'PermitNotes', 'TreeID'],
                     [3, 'Permit Number 49560', 85269],
                     [3, None, 121227],
                     [1, 'Permit Number 40897', 53918],
                     [1, 'Permit Number 49560', 85267],
                     [4, 'Permit Number 49560', 85270],
                     [226, 'Permit Number 44451', 45986]]

In [59]:
trees_species = [['SiteOrder', 'TreeID', 'Species'],
                 [3, 85269, 'Arbutus "Marina" :: Hybrid Strawberry Tree'],
                 [3, 121227, 'Sequoia sempervirens :: Coast Redwood'],
                 [1, 121399, 'Corymbia ficifolia :: Red Flowering Gum'],
                 [1, 53918, 'Prunus serrulata :: Ornamental Cherry'],
                 [4, 85270, 'Arbutus "Marina" :: Hybrid Strawberry Tree']]               

In [60]:
street_tree_info = etl.join(trees_site_permit, 
                            trees_species,
                            key = 'SiteOrder')

In [61]:
etl.lookall(street_tree_info)

+-----------+-----------------------+--------+--------+----------------------------------------------+
| SiteOrder | PermitNotes           | TreeID | TreeID | Species                                      |
+===========+=======================+========+========+==============================================+
|         1 | 'Permit Number 40897' |  53918 | 121399 | 'Corymbia ficifolia :: Red Flowering Gum'    |
+-----------+-----------------------+--------+--------+----------------------------------------------+
|         1 | 'Permit Number 40897' |  53918 |  53918 | 'Prunus serrulata :: Ornamental Cherry'      |
+-----------+-----------------------+--------+--------+----------------------------------------------+
|         1 | 'Permit Number 49560' |  85267 | 121399 | 'Corymbia ficifolia :: Red Flowering Gum'    |
+-----------+-----------------------+--------+--------+----------------------------------------------+
|         1 | 'Permit Number 49560' |  85267 |  53918 | 'Prunus serrulata

In [62]:
street_tree_info = etl.join(trees_site_permit, 
                            trees_species,
                            key='SiteOrder', 
                            presorted=True )

In [63]:
etl.lookall(street_tree_info)

+-----------+-----------------------+--------+--------+----------------------------------------------+
| SiteOrder | PermitNotes           | TreeID | TreeID | Species                                      |
+===========+=======================+========+========+==============================================+
|         3 | 'Permit Number 49560' |  85269 |  85269 | 'Arbutus "Marina" :: Hybrid Strawberry Tree' |
+-----------+-----------------------+--------+--------+----------------------------------------------+
|         3 | 'Permit Number 49560' |  85269 | 121227 | 'Sequoia sempervirens :: Coast Redwood'      |
+-----------+-----------------------+--------+--------+----------------------------------------------+
|         3 | None                  | 121227 |  85269 | 'Arbutus "Marina" :: Hybrid Strawberry Tree' |
+-----------+-----------------------+--------+--------+----------------------------------------------+
|         3 | None                  | 121227 | 121227 | 'Sequoia sempervi

#### compound keys are supported

In [64]:
tree_info = etl.join(trees_site_permit, 
                     trees_species,
                     key = ['TreeID', 'SiteOrder'])

In [65]:
etl.lookall(tree_info)

+-----------+-----------------------+--------+----------------------------------------------+
| SiteOrder | PermitNotes           | TreeID | Species                                      |
+===========+=======================+========+==============================================+
|         1 | 'Permit Number 40897' |  53918 | 'Prunus serrulata :: Ornamental Cherry'      |
+-----------+-----------------------+--------+----------------------------------------------+
|         3 | 'Permit Number 49560' |  85269 | 'Arbutus "Marina" :: Hybrid Strawberry Tree' |
+-----------+-----------------------+--------+----------------------------------------------+
|         4 | 'Permit Number 49560' |  85270 | 'Arbutus "Marina" :: Hybrid Strawberry Tree' |
+-----------+-----------------------+--------+----------------------------------------------+
|         3 | None                  | 121227 | 'Sequoia sempervirens :: Coast Redwood'      |
+-----------+-----------------------+--------+--------------

#### lookupjoin
Perform a left join, but where the key is not unique in the right-hand table, arbitrarily choose the first row and ignore others.

In [66]:
apparel_cost = [['id', 'color', 'cost'],
                [1, 'yellow', 25],
                [2, 'pink', 45],
                [3, 'purple', 30]]

In [67]:
apparel_description = [['id', 'apparel', 'size'],
                       [1, 'shirt', 'L'],
                       [1, 'shirt', 'S'],
                       [2, 'trousers', 'XS'],
                       [2, 'trousers', 'XL'],
                       [3, 'dress', 'S'],
                       [3, 'dress', 'XXS']]

In [68]:
apparel_info = etl.lookupjoin(apparel_cost, 
                              apparel_description, 
                              key = 'id')

In [69]:
etl.lookall(apparel_info)

+----+----------+------+------------+------+
| id | color    | cost | apparel    | size |
+====+==========+======+============+======+
|  1 | 'yellow' |   25 | 'shirt'    | 'L'  |
+----+----------+------+------------+------+
|  2 | 'pink'   |   45 | 'trousers' | 'XS' |
+----+----------+------+------------+------+
|  3 | 'purple' |   30 | 'dress'    | 'S'  |
+----+----------+------+------------+------+

#### crossjoin

In [70]:
apparel_cost = [['id', 'color', 'cost'],
                [1, 'yellow', 25],
                [2, 'pink', 45],
                [3, 'purple', 30]]

In [71]:
apparel_description = [['id', 'apparel', 'size'],
                       [1, 'shirt', 'L'],
                       [1, 'shirt', 'S'],
                       [2, 'trousers', 'XS'],
                       [2, 'trousers', 'XL'],
                       [3, 'dress', 'S'],
                       [3, 'dress', 'XXS'],
                       [4, 't-shirt', 'M']]

In [72]:
cross_join = etl.crossjoin(apparel_cost, apparel_description)

In [73]:
etl.lookall(cross_join)

+----+----------+------+----+------------+-------+
| id | color    | cost | id | apparel    | size  |
+====+==========+======+====+============+=======+
|  1 | 'yellow' |   25 |  1 | 'shirt'    | 'L'   |
+----+----------+------+----+------------+-------+
|  1 | 'yellow' |   25 |  1 | 'shirt'    | 'S'   |
+----+----------+------+----+------------+-------+
|  1 | 'yellow' |   25 |  2 | 'trousers' | 'XS'  |
+----+----------+------+----+------------+-------+
|  1 | 'yellow' |   25 |  2 | 'trousers' | 'XL'  |
+----+----------+------+----+------------+-------+
|  1 | 'yellow' |   25 |  3 | 'dress'    | 'S'   |
+----+----------+------+----+------------+-------+
|  1 | 'yellow' |   25 |  3 | 'dress'    | 'XXS' |
+----+----------+------+----+------------+-------+
|  1 | 'yellow' |   25 |  4 | 't-shirt'  | 'M'   |
+----+----------+------+----+------------+-------+
|  2 | 'pink'   |   45 |  1 | 'shirt'    | 'L'   |
+----+----------+------+----+------------+-------+
|  2 | 'pink'   |   45 |  1 | '

In [74]:
!pip3 install intervaltree

Using legacy 'setup.py install' for intervaltree, since package 'wheel' is not installed.
    Running setup.py install for intervaltree ... done


employee_ratings

sales target 
bonus based on target 
100-200 sales bonus x
expected min expected max


bonus 
min max bonus value

below numbers are 100,000 
200,000 and so on in dollars

In [78]:
sales_target = [['expected_sales_min', 'expected_sales_max', 'name'],
                  [100, 200, 'Alex'],
                  [200, 400, 'Alvin'],
                  [200, 500, 'Ethna'],
                  [900, 1000, 'Lily'],
                  [100, 300, 'Victor'],
                  [600, 700, 'Elliot']]

In [79]:
bonus_range = [['min', 'max', 'bonus'],
               [100, 200, 1000],
               [201, 300, 1500],
               [301, 400, 2000],
               [401, 500, 2500],
               [501, 700, 3000],
               [701, 900, 3500],
               [901, 1000, 4000]]

In [80]:
bonus_assigned = etl.intervaljoin(sales_target, 
                                  bonus_range, 
                                  lstart = 'expected_sales_min', 
                                  lstop = 'expected_sales_max',
                                  rstart = 'min', 
                                  rstop = 'max')

In [81]:
bonus_assigned.lookall()

+--------------------+--------------------+----------+-----+------+-------+
| expected_sales_min | expected_sales_max | name     | min | max  | bonus |
+====================+====================+==========+=====+======+=======+
|                100 |                200 | 'Alex'   | 100 |  200 |  1000 |
+--------------------+--------------------+----------+-----+------+-------+
|                200 |                400 | 'Alvin'  | 201 |  300 |  1500 |
+--------------------+--------------------+----------+-----+------+-------+
|                200 |                400 | 'Alvin'  | 301 |  400 |  2000 |
+--------------------+--------------------+----------+-----+------+-------+
|                200 |                500 | 'Ethna'  | 201 |  300 |  1500 |
+--------------------+--------------------+----------+-----+------+-------+
|                200 |                500 | 'Ethna'  | 301 |  400 |  2000 |
+--------------------+--------------------+----------+-----+------+-------+
|           

#### Assuming the sales people hit the max value in their ranges

In [75]:
max_bonus = etl.intervaljoin(sales_target, bonus_range, 
                             lstart = 'expected_sales_min', 
                             lstop = 'expected_sales_max',
                             lkey = 'expected_sales_max',
                             rstart = 'min',
                             rstop = 'max',
                             rkey = 'max')

In [76]:
max_bonus.lookall()

+--------------------+--------------------+----------+-----+------+-------+
| expected_sales_min | expected_sales_max | name     | min | max  | bonus |
+====================+====================+==========+=====+======+=======+
|                100 |                200 | 'Alex'   | 100 |  200 |  1000 |
+--------------------+--------------------+----------+-----+------+-------+
|                200 |                400 | 'Alvin'  | 301 |  400 |  2000 |
+--------------------+--------------------+----------+-----+------+-------+
|                200 |                500 | 'Ethna'  | 401 |  500 |  2500 |
+--------------------+--------------------+----------+-----+------+-------+
|                900 |               1000 | 'Lily'   | 901 | 1000 |  4000 |
+--------------------+--------------------+----------+-----+------+-------+
|                100 |                300 | 'Victor' | 201 |  300 |  1500 |
+--------------------+--------------------+----------+-----+------+-------+
|           

#### Add in a sales person whose expected sales ranges don't fall into one of the designated bonus ranges

In [86]:
sales_target_alt = [['expected_sales_min', 'expected_sales_max', 'name'],
                    [100, 200, 'Alex'],
                    [200, 400, 'Alvin'],
                    [200, 500, 'Ethna'],
                    [900, 1000, 'Lily'],
                    [100, 300, 'Victor'],
                    [600, 700, 'Elliot'],
                    [1500, 2000, 'Monica']]

#### Monica does not show up in the results

In [88]:
bonus_assigned = etl.intervaljoin(sales_target_alt, 
                                  bonus_range, 
                                  lstart = 'expected_sales_min', 
                                  lstop = 'expected_sales_max',
                                  rstart = 'min', 
                                  rstop = 'max')

bonus_assigned.lookall()

+--------------------+--------------------+----------+-----+------+-------+
| expected_sales_min | expected_sales_max | name     | min | max  | bonus |
+====================+====================+==========+=====+======+=======+
|                100 |                200 | 'Alex'   | 100 |  200 |  1000 |
+--------------------+--------------------+----------+-----+------+-------+
|                200 |                400 | 'Alvin'  | 201 |  300 |  1500 |
+--------------------+--------------------+----------+-----+------+-------+
|                200 |                400 | 'Alvin'  | 301 |  400 |  2000 |
+--------------------+--------------------+----------+-----+------+-------+
|                200 |                500 | 'Ethna'  | 201 |  300 |  1500 |
+--------------------+--------------------+----------+-----+------+-------+
|                200 |                500 | 'Ethna'  | 301 |  400 |  2000 |
+--------------------+--------------------+----------+-----+------+-------+
|           

In [89]:
left_join = etl.intervalleftjoin(sales_target_alt, 
                                 bonus_range, 
                                 lstart = 'expected_sales_min', 
                                 lstop = 'expected_sales_max',
                                 rstart = 'min',
                                 rstop = 'max')

In [90]:
left_join.lookall()

+--------------------+--------------------+----------+------+------+-------+
| expected_sales_min | expected_sales_max | name     | min  | max  | bonus |
+====================+====================+==========+======+======+=======+
|                100 |                200 | 'Alex'   |  100 |  200 |  1000 |
+--------------------+--------------------+----------+------+------+-------+
|                200 |                400 | 'Alvin'  |  201 |  300 |  1500 |
+--------------------+--------------------+----------+------+------+-------+
|                200 |                400 | 'Alvin'  |  301 |  400 |  2000 |
+--------------------+--------------------+----------+------+------+-------+
|                200 |                500 | 'Ethna'  |  201 |  300 |  1500 |
+--------------------+--------------------+----------+------+------+-------+
|                200 |                500 | 'Ethna'  |  301 |  400 |  2000 |
+--------------------+--------------------+----------+------+------+-------+

In [91]:
target_lookup = etl.intervallookup(sales_target, 
                                   'expected_sales_min',
                                   'expected_sales_max')

In [92]:
target_lookup.search(100)

[(100, 200, 'Alex'), (100, 300, 'Victor')]

In [93]:
target_lookup.search(100, 400)

[(100, 200, 'Alex'),
 (100, 300, 'Victor'),
 (200, 400, 'Alvin'),
 (200, 500, 'Ethna')]

In [94]:
target_lookup.search(900, 1200)

[(900, 1000, 'Lily')]

In [95]:
target_lookup = etl.intervallookup(sales_target, 
                                   'expected_sales_min',
                                   'expected_sales_max',
                                   include_stop = True,
                                   value = 'name')

In [96]:
target_lookup.search(100, 200)

['Alex', 'Victor', 'Alvin', 'Ethna']

In [97]:
target_lookup.search(600, 900)

['Elliot', 'Lily']

In [99]:
actual_sales = [['expected_sales_min', 'expected_sales_max', 'actual_sales', 'name'],
                [100, 200, 300, 'Alex'],
                [200, 400, 200, 'Alvin'],
                [200, 500, 150, 'Ethna'],
                [900, 1000, 900, 'Lily'],
                [100, 300, 400, 'Victor'],
                [600, 700, 700, 'Elliot']]

In [100]:
lookup = etl.facetintervallookup(actual_sales, 
                                 key = 'name',
                                 start = 'expected_sales_min',
                                 stop = 'expected_sales_max')

In [101]:
lookup['Alex'].search(100, 150)

[(100, 200, 300, 'Alex')]

In [102]:
lookup['Ethna'].search(100, 200)

[]